In [ ]:
%pip install transformers

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from transformers import pipeline

from nltk import sent_tokenize
import nltk
from time import time
import matplotlib.pyplot as plt
import torch

c:\Users\arnob\OneDrive\Desktop\Projects\shinobinet\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
nltk.download('punkt')

In [ ]:
subtitles_paths = sorted(glob("data/Subtitles/*.ass"))

In [ ]:
scripts = []
episode_num = []

for path in subtitles_paths:

    with open(path, "r") as file:
        lines = file.readlines()
        lines = lines[27:]

        rows = [",".join(line.split(",")[9:1]) for line in lines]

    rows = [line.replace("\\N", " ") for line in rows]
    script = " ".join(rows)

    episode = int(path.split("-")[1].split(".")[0].strip())

    scripts.append(script)
    episode_num.append(episode)

In [ ]:
df = pd.DataFrame.from_dict({'episode':episode_num, 'script':scripts})
df.head()

In [ ]:
device = 0 if torch.cuda.is_available() else "cpu"

In [ ]:
from transformers import pipeline

theme_classifier = pipeline(
    "zero-shot-classification", model="facebook/bart-large-mnli", device=device
)

In [ ]:
theme_classifier(
    "I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
    multi_label=False,
)

In [ ]:
def get_themes(script):
    script_sentences = sent_tokenize(script)

    # Batch sentences
    script_batches = []
    sentence_batch_size = 20
    for index in range(0, len(script_sentences), sentence_batch_size):
        sent = " ".join(script_sentences[index : index + sentence_batch_size])
        script_batches.append(sent)

    # Run model
    candidate_labels = [
        "friendship",
        "hope",
        "sacrifice",
        "battle",
        "self development",
        "betrayal",
        "love",
        "dialogue",
    ]
    emotion_output = theme_classifier(
        script_batches, candidate_labels, multi_label=True
    )

    # Wrangle Output
    emotions = {}
    for output in emotion_output:
        for label, score in zip(output["labels"], output["scores"]):
            if label not in emotions:
                emotions[label] = []
            emotions[label].append(score)

    emotions = {key: np.mean(np.array(value)) for key, value in emotions.items()}

    return emotions

In [ ]:
output_emotions = df["script"].apply(get_themes)

In [ ]:
emotion_df = pd.DataFrame(output_emotions.tolist())


In [ ]:
emotion_df.head()

In [ ]:
df[emotion_df.columns] = emotion_df

In [ ]:
theme_output = df.drop(["episode", "script", "dialogue"], axis=1).sum().reset_index()
theme_output.columns = ["theme", "score"]
theme_output